In [87]:
import sys
import os

# needed to be able to import from src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,PowerTransformer
from scipy import stats
from sklearn.pipeline import Pipeline

from src.functions import *

In [ ]:

dev_set_df=pd.read_csv("../data/assignment1_dev_set.csv")
val_set_df=pd.read_csv("../data/assignment1_val_set.csv")


# we replace potentially NA values in dev and val set
dev_set_df.fillna(value=0.0)
val_set_df.fillna(value=0.0)

# we remove duplicates from the dev set
shape_before=dev_set_df.shape
dev_set_df.drop_duplicates()
shape_after=dev_set_df.shape

if (shape_before != shape_after):
    print("Before removal of duplicates",shape_before)
    print("After removal of duplicates",shape_after)
else:
    print("No duplicates in the dev set")

# we remove duplicates from the val set
shape_before=val_set_df.shape
val_set_df.drop_duplicates()
shape_after=val_set_df.shape

if (shape_before != shape_after):
    print("Before removal of duplicates",shape_before)
    print("After removal of duplicates",shape_after)
else:
    print("No duplicates in the val set")

bmi=pd.DataFrame(dev_set_df["BMI"])
age=pd.DataFrame(dev_set_df["Host age"])
sex=dev_set_df["Sex"]

to_drop=['Unnamed: 0', 'Project ID','Experiment type','Sex','Host age','Disease MESH ID','BMI']

# we check for outliers
bacteria=dev_set_df.drop(labels=to_drop,axis=1)

bacteria_no_outliers=bacteria[(np.abs(stats.zscore(bacteria)) < 3).all(axis=1)]
print(bacteria.shape)
print(bacteria_no_outliers.shape)


# we need to encode the sexes to 0 and 1
sex_encoded=sex.apply(encode_sex)

bacteria_names=list(bacteria)


No duplicates in the dev set
No duplicates in the val set
(489, 134)
(131, 134)
['Acholeplasma axanthum', 'Acidaminococcus fermentans', 'Acidaminococcus intestini', 'Actinomyces lingnae', 'Akkermansia muciniphila', 'Alistipes finegoldii', 'Alistipes indistinctus', 'Alistipes obesi', 'Alistipes onderdonkii', 'Alistipes putredinis', 'Alistipes shahii', 'Alistipes sp. RMA 9912', 'Alistipes timonensis', 'Anaerofilum pentosovorans', 'Anaerofustis stercorihominis', 'Anaerostipes butyraticus', 'Anaerostipes caccae', 'Anaerotruncus colihominis', 'Bacillus megaterium', 'Bacillus nealsonii', 'Bacteroides eggerthii', 'Bacteroides fragilis', 'Bacteroides gallinarum', 'Bacteroides graminisolvens', 'Bacteroides intestinalis', 'Bacteroides massiliensis', 'Bacteroides ovatus', 'Bacteroides plebeius', 'Bacteroides salyersiae', 'Barnesiella viscericola', 'Bifidobacterium bifidum', 'Bifidobacterium pseudocatenulatum', 'Bifidobacterium saguini', 'Bifidobacterium tsurumiense', 'Bilophila wadsworthia', 'Bla

In [152]:
# we will scale our numerical data
# scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler = RobustScaler()
# scaler.fit(bacteria)

# scaled_bacteria=scaler.fit_transform(bacteria)

pipeline = Pipeline([
    ('scaler', RobustScaler()),  
    ('transformer', PowerTransformer(method='yeo-johnson'))
])

scaled_bacteria=pipeline.fit_transform(bacteria)
scaled_bacteria_df=pd.DataFrame(columns=bacteria_names,data=scaled_bacteria)
scaled_bacteria_df=scaled_bacteria_df.reindex(sorted(scaled_bacteria_df.columns),axis=1)
print(scaled_bacteria_df.shape)

(489, 134)


In [ ]:
to_be_saved_dev=pd.concat([bmi,age,sex_encoded,scaled_bacteria_df],axis=1)
print(to_be_saved_dev.shape)

(489, 137)
